# Proyecto 2: De clasificación a NER

En este proyecto utilizarán [este dataset](https://archive.ics.uci.edu/ml/datasets/Paper+Reviews) que consiste en evaluaciones de diferentes revisores sobre papers (405). Cada evaluación ha sido etiquetada con 5 clases (muy negativo, negativo, 0, neutral, positivo y muy positivo). Este proyecto consiste en realizar una clasificación de este dataset y además identificar que parte de los comentarios hacen referencia a las clases. 

## Parte 1. Clasificación sencilla de cada revisión

En esta parte tendrá que hacer un modelo de clasificación usando redes recurrentes (LSTM, GRU), para los cinco tipos de categorias.  Recuerde algunos pasos para realizar la clasificación:

1. Lectura de los datos (división en training/validation sets)
2. Preprocesamiento del texto (no es necesario ser tanto preprocesamiento como en word2vec, pero si eliminar algunos carácteres o incluso revisar si hay comentarios vacíos). 
3. Creación del vocabulario y transformación de palabras a índices (y viceversa)
4. Creación del "dataset" y "dataloader" involucrando los pasos 2 y 3.
5. Creación del modelo, involucrando embeddings.
6. Entrenar y validar (escoger optimizador Adam o AdamW, función de costo apropiada, loops de entrenamiento y validación).
7. Modelo listo para producción (modelo para predecir entrando un comentario)
8. Pequeña interfaz para predicción usando Gradio.

Realice diferentes configuraciones, tamaño del embedding, arquitecturas LSTM o GRU, varias capas, bidireccionalidad, etc. Escoja el mejor modelo. 

## Parte 2. Convirtiendo el texto a un problema de clasificación por palabras o frases (NER)

## Parte 2 A. Etiquetado 
Etiquetado de los comentarios frases a frases
Usando herramientas como [brat](https://brat.nlplab.org/), [docanno](https://doccano.herokuapp.com/) o [inception](https://inception-project.github.io/) (ver [este blog](https://dida.do/blog/the-best-free-labeling-tools-for-text-annotation-in-nlp) para un rápido resumen sobre estas aplicaciones), deberán etiquetar las frases en los comentarios con las etiquetas que correspondan a cada categoría. Un ejemplo para el primer comentario lo muestra la siguiente figura: 
![image](ejemploetiquetado.png)

Nota: El anterior ejemplo lo hice en la herramienta de inception (y exportando el archivo a WebAnno TSV v3.3).
Como pueden observar la mayoría de palabras no se etiquetarán (pertenecen a una categoría como "otros").

No deben etiquetar todo el dataset, pueden hacerlo con unas 50 (por grupo). Pero pueden intercambiarse (entre grupos) los datasets etiquetados para incrementar el número de datos (para esto es recomendable usar la misma herramienta y el mismo tipo de archivo de exportación).

### Parte 2. Clasificación y localización de las frases
Cree un modelo de clasificación por palabra (es similar al punto 2 del taller 4), donde involucren redes recurrentes. Cuando varias palabras contiguas pertenezcan a la misma etiqueta deberan unirlas para etiquetarlas como frase.  Para este modelo deben seguir los mismos pasos de la parte 1, pero cambiando un poco el modelo. 

- Desarrolle un modelo inicializando los pesos de forma aleatoria (por defecto)
- Desarrolle un modelo partiendo de los pesos de la parte 1.
- Desarrolle un modelo word2vec y posteriormente cargue estos pesos en la capa de embedding para desarrollar el modelo recurrente.Les puede ser útil [esto](https://gist.github.com/dhruvdcoder/cbb8d7967a499ba85418c18414e2cdce).
- Compare los modelos anteriores.
- Visualice la clasificación de algunas frases del conjunto de validación usando spacy o la herramienta que usaron para etiquetar.

-----
Es importante que realicen gráficos y visualizaciones que ayuden a la interpretación. No olviden ir analizando y comentando los hallazgos, y sobretodo **concluir**. El entregable es un notebook de Jupyter, debidamente presentado y comentado.

- ¿Qué diferencian encuentran con un tipo de clasificación respecto al otro?
- ¿Los pesos de una tarea ayudan a la otra?
- ¿Es posible analizar o intuir a partir de pesos o activaciones, que parte del texto está ayudando en la parte 1 a clasificar?

## Parte 3 [Opcional]. Modelo de Generación de revisiones

Esta parte es totalmente opcional, no puntua en si para el proyecto (pero si puede dar bonificación).
En esta parte debera crear un modelo de generación de texto a partir del corpus completo de las revisiones. Esto es lo que se conoce como un modelo de generación del texto. La idea es usar modelos recurrentes para generar texto. Básicamente es crear un modelo que pueda predecir las palabras del corpues teniendo en cuenta los estados anteriores (los tokens anteriores).  Se podría por ejemplo iniciar con una frase y posteriormente producir una palabra, luego se une a la frase y se sigue así de forma sucesiva. Si desean hacer esta parte, [este blog](https://www.analyticsvidhya.com/blog/2020/08/build-a-natural-language-generation-nlg-system-using-pytorch/) y [este otro blog](https://www.kdnuggets.com/2020/07/pytorch-lstm-text-generation-tutorial.html) les puede servir de ayuda.

### Tip de lectura de datos del archivo JSON

In [1]:
import pandas as pd
import json
import nltk
import re
from nltk.stem import PorterStemmer
from torch.utils.data import Dataset, DataLoader
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from torch.utils.data.dataset import random_split
import torch
import torch.nn as nn
import numpy as np
from torchtext.vocab import vocab
from collections import Counter, OrderedDict

from torch.utils.data import DataLoader

Leemos el dataset

In [2]:
f = open('reviews.json', encoding="utf8")
data = json.load(f)
f.close()

In [3]:
df = pd.json_normalize(data['paper'], record_path = ["review"])
df.head()

,confidence,evaluation,id,lan,orientation,remarks,text,timespan
0,4,1,1,es,0,,- El artículo aborda un problema contingente y...,2010-07-05
1,4,1,2,es,1,,El artículo presenta recomendaciones prácticas...,2010-07-05
2,5,1,3,es,1,,- El tema es muy interesante y puede ser de mu...,2010-07-05
3,4,2,1,es,1,,Se explica en forma ordenada y didáctica una e...,2010-07-05
4,4,2,2,es,0,,,2010-07-05


In [4]:
class TextData(Dataset):
    '''
    Dataset basico para leer los datos de tweets
    '''
    def __init__(self, filename):
        super(TextData, self).__init__()
        self.df = df[["evaluation", "text"]]
        
    def __getitem__(self, index):
        return self.df.iloc[index,0], self.df.iloc[index,1]
    
    def __len__(self):
        return len(self.df)

In [5]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    new_str = emoji_pattern.sub(r'[]', string)
    return re.sub(r'[^\w\s]', '', new_str)

In [6]:
def tokenizer(text):
    ps = PorterStemmer()
    token = nltk.RegexpTokenizer(r"\w+")
    list_text = token.tokenize(text)
    list_text = [x.lower() for x in list_text]
    list_text = [ps.stem(x) for x in list_text]
    list_text = [remove_emoji(x) for x in list_text]
    return list_text

In [7]:
df = df.drop(df[df['text'] == ""].index, axis=0)
df.reset_index(drop = True, inplace = True)
df['evaluation'] = df['evaluation'].apply(lambda x: int(x)+2)

In [8]:
ds = TextData(df)
train_dataset, valid_dataset = random_split(ds,
 [int(len(df)*0.7),len(df) - int(len(df)*0.7)], torch.manual_seed(1))

Limpiamos y tokenizamos los datos

Creamos el vocabulario

In [9]:
token_counts = Counter()

for label, line in train_dataset:
    tokens = tokenizer(line)
    token_counts.update(tokens)
 
    
print('Vocab-size:', len(token_counts))

Vocab-size: 4773


Tranformamos las palabras a índices (y viceversa)

In [10]:
sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
vocab = vocab(ordered_dict)

vocab.insert_token("<pad>", 0)
vocab.insert_token("<unk>", 1)
vocab.set_default_index(1)

Creación del "dataset" y "dataloader" involucrando los pasos 2 y 3.

In [11]:
## Paso 3-A: Defina las funciones para la transformación.

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: 1. if x == 'pos' else 0.


## Paso 3-B: función de codificación y transformación
def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), 
                                      dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    label_list = torch.tensor(label_list)
    lengths = torch.tensor(lengths)
    padded_text_list = nn.utils.rnn.pad_sequence(
        text_list, batch_first=True)
    return padded_text_list.to(device), label_list.to(device), lengths.to(device)

In [12]:
dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False, collate_fn=collate_batch)
text_batch, label_batch, length_batch = next(iter(dataloader))
print(text_batch)
print(label_batch)
print(length_batch)
print(text_batch.shape)

tensor([[  29,   73,  117,  714, 1780,  204, 1111,   11, 2523, 2524,   75,  641,
           29,   73,  922,  310, 2525,  118,   29,  332,  642, 2526,   29, 1349,
         1781,  117, 2527,  204, 1112,   29,   73, 1350,  118, 1782,  114, 1111,
         2528,   11, 2529,   29,   73,  147, 2530,  431,  466, 1112,   29,   73,
          597,  923,   11, 1783, 2531, 1784,   75,   29, 1113,  374,  310, 2532,
         1351, 1352,   10, 1114,  467, 2533, 1353, 1115, 1113,  141,  310, 2534,
         1785,  643,  283, 2535, 1786,  148,  504,  110, 1787,  374,  310, 2536,
         1788, 1354,  204,  924,  352, 1116,  118,  311, 1789,  504,  925,   11,
         1355, 2537,  468, 1356,  114, 1117, 2538,  333, 1357, 2539,  117, 1116,
         1790, 2540,  548, 2541,  808, 1791,  217, 1356,  114, 1117,  117,  310,
         1358,  227,   11, 2542, 2543,   75, 1792,  114, 2544, 2545,   11, 2546,
         1359,  467,  117, 1793,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

Creación del modelo, involucrando embeddings.

In [13]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=False, collate_fn=collate_batch)
test_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=False, collate_fn=collate_batch)

In [14]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim, 
                                      padding_idx=0) 
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, 
                           batch_first=True)
        self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)
        out, (hidden, cell) = self.rnn(out)
        out = hidden[-1, :, :]
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out
         
vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device)

Funciones de entreanmiento y evaluacion

In [81]:
def train(dataloader):
    model.train()
    total_acc, total_loss = 0, 0
    for text_batch, label_batch, lengths in dataloader:
        optimizer.zero_grad()
        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)
        loss.backward()
        optimizer.step()
        total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
        total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)
 
def evaluate(dataloader):
    model.eval()
    total_acc, total_loss = 0, 0
    with torch.no_grad():
        for text_batch, label_batch, lengths in dataloader:
            pred = model(text_batch, lengths)[:, 0]
            loss = loss_fn(pred, label_batch)
            total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
            total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)

In [82]:
loss_fn =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10 

torch.manual_seed(1)
 
for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dataloader)
    acc_valid, loss_valid = evaluate(test_dataloader)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')
 

TypeError: forward() takes 2 positional arguments but 3 were given

Entrenar y validar (escoger optimizador Adam o AdamW, función de costo apropiada, loops de entrenamiento y validación).

7. Modelo listo para producción (modelo para predecir entrando un comentario)
8. Pequeña interfaz para predicción usando Gradio.

In [17]:
#Etiquetar de 250 a 300

### Parte 2. Clasificación y localización de las frases
Cree un modelo de clasificación por palabra (es similar al punto 2 del taller 4), donde involucren redes recurrentes. Cuando varias palabras contiguas pertenezcan a la misma etiqueta deberan unirlas para etiquetarlas como frase.  Para este modelo deben seguir los mismos pasos de la parte 1, pero cambiando un poco el modelo. 

- Desarrolle un modelo inicializando los pesos de forma aleatoria (por defecto)
- Desarrolle un modelo partiendo de los pesos de la parte 1.
- Desarrolle un modelo word2vec y posteriormente cargue estos pesos en la capa de embedding para desarrollar el modelo recurrente.Les puede ser útil [esto](https://gist.github.com/dhruvdcoder/cbb8d7967a499ba85418c18414e2cdce).
- Compare los modelos anteriores.
- Visualice la clasificación de algunas frases del conjunto de validación usando spacy o la herramienta que usaron para etiquetar.

In [74]:
from unicodedata import normalize
import torch.optim as optim

In [75]:
corpus =  df['text'][250:300]

In [76]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] if w in to_ix else to_ix["<unk>"] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

def quitartildes(s):
    s = re.sub(
            r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
            normalize( "NFD", s), 0, re.I
        )
    return normalize( 'NFC', s)

def preprocess_sentence(sentence):
    return quitartildes(sentence).lower().split()

train_sentences = [preprocess_sentence(sent) for sent in corpus]

In [77]:
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent in train_sentences:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index

In [78]:
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

In [79]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [80]:
EMBEDDING_DIM = 20
HIDDEN_DIM = 10
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)